# <center>Big Data &ndash; Exercises</center>
## <center>Fall 2020 &ndash; Week 3 &ndash; ETH Zurich</center>

## Introduction
This week we will cover mostly theoretical aspects of Hadoop and HDFS and we will discuss advantages and limitations of different storage models.

#### What is Hadoop?
Hadoop provides a **distributed file system** and a
**framework for the analysis and transformation** of very **large**
data sets using the MapReduce paradigm.

Several components are part of this framework. In this course you will study HDFS, MapReduce and HBase while this exercise focuses on HDFS and storage models.


| *Component*                |*Description*  |*First developer*  |
|----------------------------------------------|---|---|
| **HDFS**                  |Distributed file system  |Yahoo!  |
| **MapReduce**   |Distributed computation framework   |Yahoo!  |
| **HBase**           | Column-oriented table service  |Powerset (Microsoft)  |
| Pig  | Dataflow language and parallel execution framework  |Yahoo!   |
| Hive            |Data warehouse infrastructure   |Facebook  |
| ZooKeeper    |Distributed coordination service   |Yahoo!  |
| Chukwa  |System for collecting management data   |Yahoo!  |
| Avro                |Data serialization system   |Yahoo! + Cloudera  |

## 1. The Hadoop Distributed File System
### 1.1 &ndash; State which of the following statements are true:

1. The HDFS namespace is a hierarchy of files and directories.

1. In HDFS, each block of the file is either 64 or 128 megabytes depending on the version and distribution of Hadoop in use, and this cannot be changed.

1. A client wanting to write a file into HDFS, first contacts the NameNode, then sends the data to it. The NameNode will write the data into multiple DataNodes in a pipelined fashion. 

1. A DataNode may execute multiple application tasks for different clients concurrently.

1. The cluster can have thousands of DataNodes and tens of thousands of HDFS clients per cluster.

1. HDFS NameNodes keep the namespace in RAM.

1. The locations of block replicas are part of the persistent checkpoint that the NameNode stores in its native file system.

1. If the block size is set to 64 megabytes, storing a file of 80 megabytes will actually require 128 megabytes of physical memory (2 blocks of 64 megabytes each). 

### 1.2 &ndash; A typical filesystem block size is 4096 bytes. How large is a block in HDFS? List at least two advantages of such choice.

### 1.3 &ndash; How does the hardware cost grow as function of the amount of data we need to store in a Distributed File System such as HDFS? Why?




### 1.4 &ndash; Single Point of Failure

1. Which component is the main single point of failure in Hadoop?

1. What is the Secondary NameNode?

### 1.5 &ndash; Scalability, Durability and Performance on HDFS
Explain how HDFS accomplishes the following requirements:

1. Scalability

1. Durability

1. High sequential read/write performance

## 2. File I/O operations and replica management.


### 2.1 &ndash; Replication policy
Assume your HDFS cluster is made of 3 racks, each containing 3 DataNodes. Assume also the HDFS is configured to use a block size of 100 megabytes and that a client is connecting from outside the datacenter (therefore no DataNode is priviledged). 

1. The client uploads a file of 150 megabytes. In the picture below, draw a possible block configuration according to the default HDFS replica policy. How many replicas are there for each block? Where are these replicas stored?

1. Can you find a with a different policy that, using the same number of replicas, improves the expected availability of a block? Does your solution show any drawbacks?

1. Referring to the picture below, assume a block is stored in Node 3, as well as in Node 4 and Node 5. If this block of data has to be processed by a task running on Node 6, which of the three replicas will be actually read by Node 6? 


<img src="https://bigdata2020exassets.blob.core.windows.net/ex03/cluster.jpg" width="500">

### 2.2 &ndash; File read and write data flow.
To get an idea of how data flows between the client interacting with HDFS, consider a diagram below which shows main components of HDFS. 

<img src="https://bigdata2020exassets.blob.core.windows.net/ex03/clientHDFS.jpg" width="600">

1. Draw the main sequence of events when a client copies a file to HDFS.
2. Draw the main sequence of events when a client reads a file from HDFS.
3. Why do you think a client writes data directly to datanodes instead of sending it through the namenode?

### 2.3 &ndash; Network topology.
HDFS estimates the network bandwidth between two nodes by their distance. The distance from a node to its parent node is assumed to be one. A distance between two nodes can be calculated by summing up their distances to their closest common ancestor. A shorter distance between two nodes means that the greater bandwidth they can utilize to transfer data. Consider a diagram of a possible hadoop cluster over two datacenters below. 

<img src="https://bigdata2020exassets.blob.core.windows.net/ex03/networkHDFS.jpg" width="700">

Calculate following distances using the distance rule explained above:
1. Node 0 and Node 1
2. Node 0 and Node 2
3. Node 1 and Node 4
4. Node 4 and Node 5
5. Node 2 and Node 3
6. Two processes of Node 1

## 3. Storage models
### 3.1 &ndash; List two differences between Object Storage and Block Storage.



### 3.2 &ndash; Compare Object Storage and Block Storage. For each of the following use cases, say which technology better fits the requirements.

1. Store Netflix movie files in such a way they are accessible from many client applications at the same time [ *Object storage | Block Storage* ]

1. Store experimental and simulation data from CERN [ *Object storage | Block Storage* ]

1. Store the auto-backups of iPhone/Android devices [ *Object storage | Block Storage* ]

## 4. Install Hadoop
You can now install Hadoop on your local computer to test HDFS in practice, we are going to try the last version  `3.3.0`. The installation on GNU/Linux is rather easy, [follow this steps](https://hadoop.apache.org/docs/r3.3.0/hadoop-project-dist/hadoop-common/SingleCluster.html). With a similar procedure you can also install on [Windows](https://kontext.tech/column/hadoop/447/install-hadoop-330-on-windows-10-step-by-step-guide) ([alternative old guide](http://wiki.apache.org/hadoop/Hadoop2OnWindows)) or [Mac OSX](https://towardsdatascience.com/installing-hadoop-on-a-mac-ec01c67b003c) ([alternative old guide](http://zhongyaonan.com/hadoop-tutorial/setting-up-hadoop-2-6-on-mac-osx-yosemite.html)).

During next recitation sessions, we will use a cluster version of HDFS on Azure. The purpose of this session is to familiarise you with shell commands and web Interface by using your local machines. 

Each Hadoop cluster is set up in one of the three supported modes:

- Local (Standalone) Mode
- Pseudo-Distributed Mode
- Fully-Distributed Mode

By default Hadoop runs in Local Mode but you should set-up it for **Pseudo-Distributed Mode**. This will allow you to run Hadoop on a single-node (your computer) simulating a distributed file system. As explained in the tutorials, to set up a Pseudo-Distributed Mode you will need to edit `etc/hadoop/core-site.xml` and `etc/hadoop/hdfs-site.xml` as follows:

* `etc/hadoop/core-site.xml`:

```xml
<configuration>
    <property>
        <name>fs.defaultFS</name>
        <value>hdfs://localhost:9000</value>
    </property>
</configuration>
```

* `etc/hadoop/hdfs-site.xml`:

```xml
<configuration>
    <property>
        <name>dfs.replication</name>
        <value>1</value>
    </property>
</configuration>
```

Hadoop distribution requires having preinstalled Java and knowing the root directory of it (Java 8 or 11 is required, more info [here](https://cwiki.apache.org/confluence/display/HADOOP/Hadoop+Java+Versions)). 
A possible root directory of your already installed Java installation on Linux is 

```
 /usr/lib/jvm/default-java
```

Try making such path available through the `JAVA_HOME` environment variable. If Hadoop still can not pick it up, add it in the `etc/hadoop/hadoop-env.sh` file.

It might be that by using `pdsh` you are required to set its authentication type to `ssh` by using `export PDSH_RCMD_TYPE=ssh` (more info [here](https://stackoverflow.com/questions/42756555/permission-denied-error-while-running-start-dfs-sh))

Once you have formatted your filesystem (`$ bin/hdfs namenode -format`) and started the NameNode daemon (`$ sbin/start-dfs.sh`) you should be able to browse `http://localhost:9870/` and visualize the web interface of the daemon which should look similar to the following:

<img src="https://bigdata2020exassets.blob.core.windows.net/ex03/hadoop.png" width="1100">

In the `Datanodes` tab you should see a single operating datanode.

### 4.1 &ndash; Upload a file into HDFS

Pick an image file in your computer (or you can also download a random one) and try to upload it to HDFS. You may need to create an empty directory before uploading.

1. Which command do you use to upload from the local file system to HDFS?

1. Which information can you find if you use `Utilities -> Browse the file system` in the daemon web interface?

### 4.2 &ndash; Local File System

Now take a look at the documented default values of `hdfs-default.xml` and `core-default.xml` and locate the file that you have just uploaded into HDFS.

- http://hadoop.apache.org/docs/r3.3.0/hadoop-project-dist/hadoop-hdfs/hdfs-default.xml
- http://hadoop.apache.org/docs/r3.3.0/hadoop-project-dist/hadoop-common/core-default.xml


1. Is the image still accessible with a normal image viewer? 

1. What is the path of the image relative to the HDFS filesystem?

1. What is the path of the image relative to your OS filesystem?

### 4.3 &ndash; Local File System

Now try to upload a file that is >150MB. On Unix-based system you can also generate such a file filled with zero using:

```
$ dd if=/dev/zero of=zeros.dat bs=1M count=150
```

1. How many blocks the file is split into?

### 4.4 Stop HDFS

Once you are done experimenting you can stop HDFS by running `$ sbin/stop-dfs.sh`